<a href="https://colab.research.google.com/github/Ludvins/Practicas_PDGE/blob/master/CUDA/Multiplicacion_matrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Luis Antonio Ortega Andrés    
Antonio Coín Castro*

# Ejercicio opcional: multiplicación de matrices

Consideramos el problema de multiplicar dos matrices cuadradas $A$ y $B$ de dimensiones $N\times N$, obteniendo como resultado una matriz $C$, también $N\times N$.

## Multiplicación en CPU

Realizamos la multiplicación en CPU, cuya eficiencia es $O(N^3)$. 

In [115]:
%%writefile matmul_cpu.cu

#include <stdio.h>

#define N 64

void matrixMultCPU(float a[N][N], float b[N][N], float c[N][N]) {
  int i, j, k;
  for (i=0; i < N; i++) {
    for (j = 0; j < N; j++) {
      float sum = 0;
      for (k = 0; k < N; k++) {
        sum += a[i][k] * b[k][j];
      }
      c[i][j] = sum;
    }
  }
}

int main() {
  float a[N][N], b[N][N], c[N][N];

  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  /* inicializando variables con datos*/
  for (int i = 0; i < N; i++) {
    for (int j = 0; j < N; j++) {
      a[i][j] = 1.0f*j;
      b[i][j] = 1.0f*j;
    }
  }

  int nIter=1000;
  cudaEventRecord(start);
  for (int i = 0; i < nIter; i++)
    matrixMultCPU(a, b, c);

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  float msecTotal = 0.0f;
  cudaEventElapsedTime(&msecTotal, start, stop);

  // Comprueba resultados
  int errores = 0;

  // imprimiendo
  for (int y = 0; y < N; y++) {
    for (int x = 0; x < N; x++) {
        if (c[y][x] != (N*(N-1))/2 *x)
          errores++;
    }
  }

  if (errores == 0)
    printf("Resultado correcto.\n");
  else
    printf("Errores: %d.\n", errores);

  float msecPerKernelExecution = msecTotal / nIter;
  double flopsPerMMull = 2.0 * N * N * N;
  double gigaFlops = (flopsPerMMull * 1.0e-9f) /
    (msecPerKernelExecution / 1000.0f);

  printf("Tiempo medio de ejecución: %f ms\n", msecPerKernelExecution);
  printf("GFLOPS: %f\n", gigaFlops);
  return 0;
}

Overwriting matmul_cpu.cu


In [116]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true matmul_cpu.cu -o matmul_cpu -lcudadevrt
!./matmul_cpu

Resultado correcto.
Tiempo medio de ejecución: 0.794934 ms
GFLOPS: 0.659537


## Multiplicación en GPU

Pasamos ahora a realizar la misma implementación pero en GPU. La adaptamos para un tamaño $N$ arbitrario, empleando siempre bloques de $8\times 8$ threads (para aprovechar al máximo el paralelismo, como ya comentamos en el ejercicio sobre la suma de matrices), y un número de bloques igual a $(N+7)/8$. Diferenciamos según si se utiliza memoria compartida o no.

### Sin usar memoria compartida

Implementamos en primer lugar el algoritmo más sencillo. Establecemos una estructura bidimensional de bloques y threads, y el thread con identificador $(i, j)$ (en coordenadas de grid) procesa la fila $j$-ésima y la columna $i$-ésima para obtener el elemento $C_{ij}$. De esta forma obtenemos del orden de $N^2$ threads, donde cada uno realiza $O(N)$ operaciones.

In [144]:
%%writefile matmul_gpu_v1.cu

#include <stdio.h>

#define N 64
#define TPB 8

// Computa la multiplicación de matrices en GPU sin memoria compartida
__global__ void matrixMultGPU(int n, float *a, float *b, float *c) {
  int k;
  float sum = 0;
  int col = threadIdx.x + blockDim.x * blockIdx.x;
  int row = threadIdx.y + blockDim.y * blockIdx.y;
  
  if (row < n && col < n) {
    for (k = 0; k < n; k++) {
      sum += a[row * n + k] * b[k * n + col];
    }
    c[row * n + col] = sum;
  }
}

int main() {
  float a[N][N], b[N][N], c[N][N];
  float *dev_a, *dev_b, *dev_c;
  int size = N * N * sizeof(float);

  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaMalloc((void **) &dev_a, size);
  cudaMalloc((void **) &dev_b, size);
  cudaMalloc((void **) &dev_c, size);

  /* inicializando variables con datos*/
  for (int i = 0; i < N; i++) {
    for (int j = 0; j < N; j++) {
      a[i][j] = 1.0f*j;
      b[i][j] = 1.0f*j;
    }
  }

  cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(dev_b, b, size, cudaMemcpyHostToDevice);

  // Tamaño de grid y bloque
  dim3 dimGrid((N+TPB-1)/TPB, (N+TPB-1)/TPB);
  dim3 dimBlock(TPB, TPB);

  int nIter=1000;
  cudaEventRecord(start);
  for (int i=0; i <nIter; i++)
    matrixMultGPU<<<dimGrid, dimBlock>>>(N, dev_a, dev_b, dev_c);
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  cudaMemcpy(c, dev_c, size, cudaMemcpyDeviceToHost);

  float msecTotal = 0.0f;
  cudaEventElapsedTime(&msecTotal, start, stop);

  // Comprueba resultados
  int errores = 0;
  // imprimiendo
  for (int y = 0; y < N; y++) {
    for (int x = 0; x < N; x++) {
        if (c[y][x] != (N*(N-1))/2 *x)
          errores++;
      //printf("[%d][%d]=%d ", y, x, c[y][x]);
    }
    //printf("\n");
  }

  printf("Resultado ");
  if (errores == 0)
    printf("correcto.\n");
  else
    printf("incorrecto. Errores: %d\n", errores);

  float msecPerKernelExecution = msecTotal / nIter;
  double flopsPerMMull = 2.0 * N * N * N;
  double gigaFlops = (flopsPerMMull * 1.0e-9f) /
    (msecPerKernelExecution / 1000.0f);

  printf("GFLOPS: %f\n", gigaFlops);

  cudaFree(dev_a);
  cudaFree(dev_b);
  cudaFree(dev_c);

  return 0;
}

Overwriting matmul_gpu_v1.cu


In [145]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true matmul_gpu_v1.cu -o matmul_gpu_v1 -lcudadevrt
!nvprof ./matmul_gpu_v1

==5118== NVPROF is profiling process 5118, command: ./matmul_gpu_v1
Resultado correcto.
GFLOPS: 56.636388
==5118== Profiling application: ./matmul_gpu_v1
==5118== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.89%  7.9593ms      1000  7.9590us  7.8720us  13.344us  matrixMultGPU(int, float*, float*, float*)
                    0.07%  5.5680us         2  2.7840us  2.6560us  2.9120us  [CUDA memcpy HtoD]
                    0.04%  2.8800us         1  2.8800us  2.8800us  2.8800us  [CUDA memcpy DtoH]
      API calls:   95.15%  193.74ms         2  96.868ms     940ns  193.73ms  cudaEventCreate
                    2.85%  5.8013ms      1000  5.8010us  4.3010us  30.375us  cudaLaunchKernel
                    1.54%  3.1379ms         1  3.1379ms  3.1379ms  3.1379ms  cudaEventSynchronize
                    0.17%  356.31us         1  356.31us  356.31us  356.31us  cuDeviceTotalMem
                    0.09%  174.00us        97 

Vemos que se realiza adecuadamente el producto y no se producen errores. Además, el tiempo medio por ejecución es de tan solo 8 us, muy por debajo del tiempo medio de ejecución en CPU.

## Memoria compartida con optimizaciones adicionales

Pasamos ahora a optimizar la forma de cálculo del producto. En concreto hacemos tres optimizaciones:

- Utilizamos memoria compartida para el acceso a los datos. De esta forma, definiremos "mosaicos" para hacer el producto de matrices por bloques, de forma independiente y optimizando el acceso a memoria. Así conseguimos que las matrices $A$ y $B$ se carguen solo $N/8$ veces en memoria, al contrario que antes que se cargaban $N$ veces. Necesitamos usar `__syncthreads()` para asegurarnos de que todos los datos necesarios han sido cargados.
- Accedemos a los mosaicos de forma traspuesta para aumentar la eficiencia en memoria, debido a que se guardan en "column-major".
- Desenrrollamos bucles para optimización del compilador. Utilizamos la directiva del precompilador `#pragma unroll`.

In [154]:
%%writefile matmul_gpu_v2.cu

#include <stdio.h>

#define N 64
#define TPB 8

__global__ void matrixMultGPU2(int n, float* A, float* B, float* C) {
    float sum = 0;
    int tile;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int i = blockIdx.x * blockDim.x + tx;
    int j = blockIdx.y * blockDim.y + ty;

    if (i<n && j < n) {
      // Mosaicos en memoria de bloque
      __shared__ float As[TPB][TPB];
      __shared__ float Bs[TPB][TPB];

      // Recorre los mosaicos de A y B necesarios para computar la submatriz de C
      for (tile = 0; tile < ((n+TPB-1)/TPB); tile++){
          // Carga los mosaicos de A y B en paralelo (y de forma traspuesta)
          As[ty][tx] = A[(i * n) + (ty + (tile*TPB))];
          Bs[ty][tx] = B[((tx + (tile * TPB))*n) + j];

          __syncthreads();

          // Computa los resultados para la submatriz de C (también traspuestos)
  #pragma unroll
          for (int k = 0; k < TPB; k++)
            sum += As[k][tx] * Bs[ty][k];

          __syncthreads();
      }
      // Escribe en paralelo los resultados obtenidos por el bloque
      C[i * n + j] = sum;
    }
}

int main() {
  float a[N][N], b[N][N], c[N][N];
  float *dev_a, *dev_b, *dev_c;
  int size = N * N * sizeof(float);

  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaMalloc((void **) &dev_a, size);
  cudaMalloc((void **) &dev_b, size);
  cudaMalloc((void **) &dev_c, size);

  /* inicializando variables con datos*/
  for (int i = 0; i < N; i++) {
    for (int j = 0; j < N; j++) {
      a[i][j] = 1.0f*j;
      b[i][j] = 1.0f*j;
    }
  }

  cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(dev_b, b, size, cudaMemcpyHostToDevice);

  // Tamaño de grid y bloque
  dim3 dimGrid((N+TPB-1)/TPB, (N+TPB-1)/TPB);
  dim3 dimBlock(TPB, TPB);

  int nIter=1000;
  cudaEventRecord(start);
  for (int i=0; i <nIter; i++) {
    matrixMultGPU2<<<dimGrid, dimBlock>>>(N, dev_a, dev_b, dev_c);
  }
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  cudaMemcpy(c, dev_c, size, cudaMemcpyDeviceToHost);

  float msecTotal = 0.0f;
  cudaEventElapsedTime(&msecTotal, start, stop);

  // Comprueba resultados
  int errores = 0;
  // imprimiendo
  for (int y = 0; y < N; y++) {
    for (int x = 0; x < N; x++) {
        if (c[y][x] != ((N*(N-1))/2) *x)
          errores++;
      //printf("[%d][%d]=%d ", y, x, c[y][x]);
    }
    //printf("\n");
  }

  printf("Resultado ");
  if (errores == 0)
    printf("correcto.\n");
  else
    printf("incorrecto. Errores: %d\n", errores);

  float msecPerKernelExecution = msecTotal / nIter;
  double flopsPerMMull = 2.0 * N * N * N;
  double gigaFlops = (flopsPerMMull * 1.0e-9f) /
    (msecPerKernelExecution / 1000.0f);

  printf("GFLOPS: %f\n", gigaFlops);

  cudaFree(dev_a);
  cudaFree(dev_b);
  cudaFree(dev_c);

  return 0;
}

Overwriting matmul_gpu_v2.cu


In [155]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true matmul_gpu_v2.cu -o matmul_gpu_v2 -lcudadevrt
!nvprof ./matmul_gpu_v2

==5427== NVPROF is profiling process 5427, command: ./matmul_gpu_v2
Resultado correcto.
GFLOPS: 54.960312
==5427== Profiling application: ./matmul_gpu_v2
==5427== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.90%  8.2816ms      1000  8.2810us  8.2230us  12.992us  matrixMultGPU2(int, float*, float*, float*)
                    0.07%  5.5680us         2  2.7840us  2.6240us  2.9440us  [CUDA memcpy HtoD]
                    0.04%  2.9120us         1  2.9120us  2.9120us  2.9120us  [CUDA memcpy DtoH]
      API calls:   95.67%  225.59ms         2  112.80ms  1.0630us  225.59ms  cudaEventCreate
                    2.40%  5.6589ms      1000  5.6580us  4.3970us  34.894us  cudaLaunchKernel
                    1.52%  3.5740ms         1  3.5740ms  3.5740ms  3.5740ms  cudaEventSynchronize
                    0.17%  399.39us         1  399.39us  399.39us  399.39us  cuDeviceTotalMem
                    0.07%  169.88us         3

Podemos probar a incrementar el tamaño de las matrices, **modificarlo para que no sean potencia de 2**, e incluso variar el número de hilos por bloque. Comprobamos que en todo caso el resultado obtenido con esta versión sigue siendo correcto.

In [156]:
!sed -i '/#define N/c\#define N 60' matmul_gpu_v2.cu
!sed -i '/#define TPB/c\#define TPB 32' matmul_gpu_v2.cu
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true matmul_gpu_v2.cu -o matmul_gpu_v2 -lcudadevrt
!nvprof ./matmul_gpu_v2

==5479== NVPROF is profiling process 5479, command: ./matmul_gpu_v2
Resultado incorrecto. Errores: 3572
GFLOPS: 16.821465
==5479== Profiling application: ./matmul_gpu_v2
==5479== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.97%  24.417ms      1000  24.416us  24.192us  24.704us  matrixMultGPU2(int, float*, float*, float*)
                    0.02%  5.2800us         2  2.6400us  2.4960us  2.7840us  [CUDA memcpy HtoD]
                    0.01%  2.7520us         1  2.7520us  2.7520us  2.7520us  [CUDA memcpy DtoH]
      API calls:   89.26%  218.53ms         2  109.27ms  1.0810us  218.53ms  cudaEventCreate
                    7.98%  19.538ms         1  19.538ms  19.538ms  19.538ms  cudaEventSynchronize
                    2.38%  5.8256ms      1000  5.8250us  4.5150us  29.132us  cudaLaunchKernel
                    0.15%  375.16us         1  375.16us  375.16us  375.16us  cuDeviceTotalMem
                    0.07%  17

## Medición de tiempos

Tiempos medios de ejecución, con/sin optimizaciones, TPB=8 y TPB=32.

### Otro tipo de dato

Usamos matrices de tipo `double` en vez de `float`.

### Comparación con el programa de ejemplo de CUDA

In [104]:
!cat /usr/local/cuda/samples/0_Simple/matrixMul/matrixMul.cu

/**
 * Copyright 1993-2015 NVIDIA Corporation.  All rights reserved.
 *
 * Please refer to the NVIDIA end user license agreement (EULA) associated
 * with this source code for terms and conditions that govern your use of
 * this software. Any use, reproduction, disclosure, or distribution of
 * this software and related documentation outside the terms of the EULA
 * is strictly prohibited.
 *
 */

/**
 * Matrix multiplication: C = A * B.
 * Host code.
 *
 * This sample implements matrix multiplication which makes use of shared memory
 * to ensure data reuse, the matrix multiplication is done using tiling approach.
 * It has been written for clarity of exposition to illustrate various CUDA programming
 * principles, not with the goal of providing the most performant generic kernel for matrix multiplication.
 * See also:
 * V. Volkov and J. Demmel, "Benchmarking GPUs to tune dense linear algebra,"
 * in Proc. 2008 ACM/IEEE Conf. on Supercomputing (SC '08),
 * Piscataway, NJ: IEEE Press, 